In [ ]:
! pip install --upgrade langchain langchain-google-genai "langchain[docarray]" faiss-cpu pypdf

In [ ]:
import os
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, Markdown
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.document import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from google.colab import userdata
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [3]:
def load_model(model_natm):
    if model_natm == "gemini-pro":
        llm = ChatGoogleGenerativeAI(model="gemini-pro")

    else:
        llm = ChatGoogleGenerativeAI(model="gemini-pro-vision")

    return llm

In [ ]:
model_for_text = load_model("gemini-pro")

In [ ]:
model_for_text.invoke("Tell me about AI").content

In [ ]:
model_for_text(
    [
        HumanMessage(content="Tell me about AI")
    ]
).content

Method for Loading Images

In [ ]:
def get_image(url, file_name, extension):
    content = requests.get(url).content

    with open(f"/content/{file_name}.{extension}", "wb") as f:
        f.write(content)

    image = Image.open(f"/content/{file_name}.{extension}")
    image.show()

    return image

In [ ]:
image = get_image("use any image URL here")

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
vision_model = load_model("gemini-pro-vision")

In [ ]:
prompt = "Provide me summary of the image in 5 words"

In [ ]:
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": prompt,
        },
        {
            "type": "image_url", "image_url": image
        }
    ]
)

In [ ]:
print(vision_model.invoke([message]).content)

In [ ]:
data_loader = TextLoader("/content/nike_shoes.txt")
print(data_loader.load()[0].page_content)

In [ ]:
text = data_loader.load()[0].page_content

In [ ]:
def get_text_chunks_langchain(text):

    text_splitter = CharacterTextSplitter(chunk_size=20, chunk_overlap=10)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]

    return docs

In [ ]:
docs = get_text_chunks_langchain(text)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [ ]:
vector_store = FAISS.from_documents(docs, embedding=embeddings)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
retriever.invoke("Nike slide/sandal.")

In [ ]:
vision_model = load_model("gemini-pro-vision")

In [ ]:
llm_model =load_model("gemini-pro")

In [ ]:
template = """ 

'''
{context}
'''

{query}

Provide berif information and store location

"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template=template)

In [ ]:
RAG_CHAIN = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm_model
    | StrOutputParser()
)

In [ ]:
result = RAG_CHAIN.invoke("Provide detail about nike sandal")

In [ ]:
display(Markdown(result))

Include Vision Model to RAG System

In [ ]:
FULL_CHAIN = (
    RunnablePassthrough() | vision_model | StrOutputParser() | RAG_CHAIN
)

In [ ]:
url = "image_of_nike_shoe"

In [ ]:
image = get_image(url, "nike3", "png")

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
message = HumanMessage(

    content=[
        {
        # exact answers with respect to data
        "type": "text",
        "text": "Provide information on givin image and model of given image."
        },

        # You can optionally provide text parts
        {"type": "image_url", "image_url":image},
    ]
)

In [ ]:
result = FULL_CHAIN.invoke([message])

In [ ]:
display(Markdown(result))